In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import joblib
from stock_price_api import *

In [ ]:
model = load_model('lstm_model.h5')
scaler = joblib.load('scaler.gz')

In [2]:
data_dict, today = get_data(day_before=100, order='d')
df= format_data(data_dict)
df = df.head(20)
print(len(df))
print(df)

2024-04-05 2024-07-14
{'2024-07-12': {'open': 228.92, 'high': 232.64, 'low': 228.68, 'close': 230.54, 'adjusted_close': 230.54, 'volume': 53008200}, '2024-07-11': {'open': 231.39, 'high': 232.39, 'low': 225.77, 'close': 227.57, 'adjusted_close': 227.57, 'volume': 64710600}, '2024-07-10': {'open': 229.3, 'high': 233.08, 'low': 229.25, 'close': 232.98, 'adjusted_close': 232.98, 'volume': 62627700}, '2024-07-09': {'open': 227.93, 'high': 229.4, 'low': 226.37, 'close': 228.68, 'adjusted_close': 228.68, 'volume': 48076100}, '2024-07-08': {'open': 227.09, 'high': 227.85, 'low': 223.25, 'close': 227.82, 'adjusted_close': 227.82, 'volume': 59085900}, '2024-07-05': {'open': 221.65, 'high': 226.45, 'low': 221.65, 'close': 226.34, 'adjusted_close': 226.34, 'volume': 60412400}, '2024-07-03': {'open': 220, 'high': 221.55, 'low': 219.03, 'close': 221.55, 'adjusted_close': 221.55, 'volume': 37369800}, '2024-07-02': {'open': 216.15, 'high': 220.38, 'low': 215.1, 'close': 220.27, 'adjusted_close': 220.

In [3]:
def get_scaled_data(df: pd.DataFrame, scaler: MinMaxScaler, data_size:int=500, path:str=r"..\Data\AAPL_stock_prices.csv", delimeter: str = ',', from_end: bool = True, date_column: str = 'Date', target_column: str = 'Close'):
    target_column_index = df.columns.tolist().index(target_column)
    scaled_data = scaler.fit_transform(df)
    return scaled_data, target_column_index

In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data, target_column_index = get_scaled_data(df=df, scaler=scaler)

In [6]:
def create_dataset(data: np.ndarray, time_step: int=10):
    X = []
    for i in range(len(data) - time_step):
        # Define the range of input sequences
        end_ix = i + time_step
        
        # Ensure that the dataset is within bounds
        if end_ix > len(data)-1:
            break
            
        # Extract input and output parts of the pattern
        seq_x = data[i:end_ix]
        
        # Append the parts
        X.append(seq_x)
    return np.array(X), data.shape[1], time_step

X, feature_number, time_step = create_dataset(data=scaled_data)
print(len(X))
print(len(df))
print(X[:2])

10
20
[[[0.8956485  0.9797235  0.97484554 0.90427619 0.90427619 0.07480641]
  [1.         0.96820276 0.84642542 0.78775991 0.78775991 0.13078494]
  [0.91170258 1.         1.         1.         1.         0.12082137]
  [0.85382341 0.83041475 0.8729038  0.83130639 0.83130639 0.05121367]
  [0.81833545 0.75898618 0.73521624 0.79756767 0.79756767 0.10387914]
  [0.58850866 0.69447005 0.66460724 0.73950569 0.73950569 0.11022446]
  [0.51880017 0.46866359 0.548985   0.55158886 0.55158886 0.        ]
  [0.35614702 0.41474654 0.37555163 0.50137309 0.50137309 0.09890572]
  [0.18462188 0.28248848 0.23521624 0.36327972 0.36327972 0.11017902]
  [0.34009294 0.21612903 0.16372462 0.12279325 0.12279325 0.21608493]]

 [[1.         0.96820276 0.84642542 0.78775991 0.78775991 0.13078494]
  [0.91170258 1.         1.         1.         1.         0.12082137]
  [0.85382341 0.83041475 0.8729038  0.83130639 0.83130639 0.05121367]
  [0.81833545 0.75898618 0.73521624 0.79756767 0.79756767 0.10387914]
  [0.5885086

In [ ]:
predicted_data = model.predict(X)
print(predicted_data)

In [11]:
df = df.head(20)
print(len(df)) # 20
# Predicted data (replace with your actual predictions)
predicted_data = np.array([230.0, 231.0, 232.0, 233.0, 234.0, 235.0, 236.0, 237.0, 238.0, 239.0])

# Create the new DataFrame
new_df = df[['Close']].iloc[10:20].copy()
new_df['Predicted_close'] = predicted_data

last_prediction = pd.DataFrame({'Close': [np.nan], 'Predicted_close': [predicted_data[-1]]}, index=[f"{today} 00:00:00"])
new_df = pd.concat([last_prediction, new_df])

print(new_df)

#ORDER ??

20
                           Close  Predicted_close
2024-07-14 00:00:00          NaN     239.00000000
2024-06-27 00:00:00 214.10000000     230.00000000
2024-06-26 00:00:00 213.25000000     231.00000000
2024-06-25 00:00:00 209.07000000     232.00000000
2024-06-24 00:00:00 208.14000000     233.00000000
2024-06-21 00:00:00 207.49000000     234.00000000
2024-06-20 00:00:00 209.68000000     235.00000000
2024-06-18 00:00:00 214.29000000     236.00000000
2024-06-17 00:00:00 216.67000000     237.00000000
2024-06-14 00:00:00 212.49000000     238.00000000
2024-06-13 00:00:00 214.24000000     239.00000000
